In [18]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense, Lambda, BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils, load_img, img_to_array

In [3]:
from sklearn.datasets import load_files
import numpy as np
from tqdm import tqdm

## Load images

In [4]:
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 102)
    return files, targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('./Data/Datasplit/train')
valid_files, valid_targets = load_dataset('./Data/Datasplit/valid')
test_files, test_targets = load_dataset('./Data/Datasplit/test')

In [5]:
train_targets.shape

(1020, 102)

## Convert images to float

In [6]:
def path_to_tensor(img_path):
    img = load_img(img_path, target_size=(224, 224))
    img = img_to_array(img)      
    img = np.expand_dims(img, axis=0)  # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3)
    img = preprocess_input(img)
    return img

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [7]:
train_tensors = paths_to_tensor(train_files)
valid_tensors = paths_to_tensor(valid_files)
test_tensors = paths_to_tensor(test_files)

100%|███████████████████████████████████████| 6149/6149 [01:03<00:00, 96.58it/s]


In [8]:
train_tensors.shape, valid_tensors.shape, test_tensors.shape

((1020, 224, 224, 3), (6149, 224, 224, 3))

## Use Resnet50 weights

In [12]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
#Loading the ResNet50 model with pre-trained ImageNet weights
resnet = tf.keras.applications.ResNet50(weights='imagenet',include_top=False,input_tensor=inputs)

In [13]:
len(resnet.layers)

175

In [14]:
for layer in resnet.layers[:170]: layer.trainable = False

In [30]:
model = Sequential()
model.add(Lambda(lambda x: tf.image.resize(x,(224, 224))))
model.add(resnet)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(102, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), 
              metrics=['accuracy'])
model.fit(train_tensors, train_targets, batch_size=32, epochs=10, verbose=0, 
          validation_data=(valid_tensors, valid_targets), shuffle=True)

In [33]:
model.evaluate(train_tensors, train_targets)
model.evaluate(valid_tensors, valid_targets)
model.evaluate(test_tensors, test_targets);

193/193 [==============================] - 688s 4s/step - loss: 3.9579 - accuracy: 0.2415


In [32]:
flower_preds = [np.argmax(model.predict(np.expand_dims(tensor, axis=0), verbose=0)) for tensor in test_tensors]
test_accuracy = 100*np.sum(np.array(flower_preds)==np.argmax(test_targets, axis=1))/len(flower_preds)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 24.1503%
